In [1]:
# To start with, we use the default stack
!zenml init

# We also need to connect to a remote ZenML Instance
!zenml connect --url https://1cf18d95-zenml.cloudinfra.zenml.io

# Now we set the stack
!zenml stack set local-mlflow-stack

NumExpr defaulting to 8 threads.
Found existing ZenML repository at path 
'/home/apenner/PycharmProjects/zenml-projects/stack-showcase'.
⠋ Initializing ZenML repository at 
/home/apenner/PycharmProjects/zenml-projects/stack-showcase.
⠋ Initializing ZenML repository at 
/home/apenner/PycharmProjects/zenml-projects/stack-showcase.

NumExpr defaulting to 8 threads.
Connecting to: 'https://1cf18d95-zenml.cloudinfra.zenml.io'...
The ZenML global configuration version (0.51.0) is higher than the version of ZenML currently being used (0.50.0). Read more about this issue and how to solve it here: https://docs.zenml.io/user-guide/advanced-guide/environment-management/global-settings-of-zenml#version-mismatch-downgrading
If your browser did not open automatically, please open the following URL into your browser to proceed with the authentication:

https://1cf18d95-zenml.cloudinfra.zenml.io/devices/verify?device_id=fd53a8cf-9831-4b53-9d9e-4ff82142051f&user_code=4494f5b928c8c79f5b9b9322cc932521

S

In [2]:
# Do the imports at the top

import random
from zenml import ExternalArtifact, pipeline 
from zenml.client import Client
from zenml.logger import get_logger
from uuid import UUID

import os
from typing import Optional, List

from zenml import pipeline

from pipelines import feature_engineering

from steps import (
    data_loader,
    data_preprocessor,
    data_splitter,
    model_evaluator,
    model_trainer,
    inference_predict,
    inference_preprocessor
)

logger = get_logger(__name__)

client = Client()

NumExpr defaulting to 8 threads.
The ZenML global configuration version (0.51.0) is higher than the version of ZenML currently being used (0.50.0). Read more about this issue and how to solve it here: https://docs.zenml.io/user-guide/advanced-guide/environment-management/global-settings-of-zenml#version-mismatch-downgrading


# Start local with a simple training pipeline

Below you can see what the pipeline looks like. We will start by running this locally on the default-stack. This means the data between the steps is stored locally and the compute is also local.

<img src="_assets/default_stack.png" alt="Drawing" style="width: 800px;"/>

In [3]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from zenml import step
from zenml.logger import get_logger

logger = get_logger(__name__)

# Here is what one of the steps in the pipeline looks like. Simple python function that just needs the `@step` decorator.

@step
def data_loader() -> pd.DataFrame:
    """Dataset reader step."""
    dataset = load_breast_cancer(as_frame=True)
    inference_size = int(len(dataset.target) * 0.05)
    dataset: pd.DataFrame = dataset.frame
    dataset.reset_index(drop=True, inplace=True)
    logger.info(f"Dataset with {len(dataset)} records loaded!")

    return dataset


In [4]:
# Here's an example of what this function returns

data_loader()

Dataset with 569 records loaded!
Dataset with 569 records loaded!


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [5]:
from zenml import pipeline

@pipeline
def training(
    train_dataset_id: Optional[UUID] = None,
    test_dataset_id: Optional[UUID] = None,
    min_train_accuracy: float = 0.0,
    min_test_accuracy: float = 0.0,
):
    """Model training pipeline."""
    # Execute Feature Engineering Pipeline
    dataset_trn, dataset_tst = feature_engineering()

    model = model_trainer(
        dataset_trn=dataset_trn,
    )

    model_evaluator(
        model=model,
        dataset_trn=dataset_trn,
        dataset_tst=dataset_tst,
        min_train_accuracy=min_train_accuracy,
        min_test_accuracy=min_test_accuracy,
    )


In [7]:
pipeline_args = {"enable_cache": True}

# Model Version config
pipeline_args["config_path"] = os.path.join("configs", "training.yaml")
fe_t_configured = training.with_options(**pipeline_args)

In [8]:
fe_t_configured()

Initiating a new run for the pipeline: training.
Reusing registered version: (version: 17).
New model version 11 was created.
Executing a new run.
Using user: alexej@zenml.io
Using stack: local-mlflow-stack
  model_registry: mlflow
  experiment_tracker: mlflow
  artifact_store: default
  orchestrator: default
Step data_loader has started.
Dataset with 541 records loaded!
Step data_loader has finished in 16.975s.
Step data_splitter has started.
Step data_splitter has finished in 24.058s.
Step data_preprocessor has started.
Step data_preprocessor has finished in 32.030s.
Caching disabled explicitly for model_trainer.
Step model_trainer has started.
Training model DecisionTreeClassifier()...
/home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages/zenml/integrations

# Let's outsource some compute to Sagemaker!

Let's farm some compute to AWS with a training job with a certain number of CPUs and Memory. This can easily be done without and changes to the actual implementation of the pipeline. 














<img src="_assets/local_sagmaker_so_stack.png" alt="Drawing" style="width: 800px;"/>

In [9]:
# This pip installs the requirements locally
!zenml integration install aws s3 mlflow -y

# This changes the active stack
!zenml stack set local-sagemaker-step-operator-stack

NumExpr defaulting to 8 threads.
⠏ Installing integrations...Requirement already satisfied: sagemaker==2.117.0 in /home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages (2.117.0)
⠋ Installing integrations...Requirement already satisfied: google-pasta in /home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages (from sagemaker==2.117.0) (0.2.0)
⠹ Installing integrations...Collecting attrs<23,>=20.3.0
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
⠸ Installing integrations...Collecting argparse
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
⠼ Installing integrations...Requirement already satisfied: scipy<2 in /home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages (from mlflow<=2.6.0,>=2.1.1) (1.11.3)
⠴ Installing integrations...Requirement already satisfied: uvloop in /home/apenner/.pyenv/versions/3.9.13/envs/demo/lib/python3.9/site-packages (from mlserver>=1.3.3) (0.19.0)
⠦ Installing integrations...Requirement a

In [10]:
from zenml.config import DockerSettings

# The actual code will stay the same, all that needs to be done is some configuration

step_args = {}

# We configure which step operator should be used
step_args["step_operator"] = "sagemaker-eu"

# M5 Large is what we need for this big data!
step_args["settings"] = {"step_operator.sagemaker": {"estimator_args": {"instance_type" : "ml.m5.large"}}}

# Update the step. We could also do this in YAML
model_trainer = model_trainer.with_options(**step_args)

docker_settings = DockerSettings(
    requirements=[
        "pyarrow",
    ],
)

pipeline_args = {"enable_cache": False, "settings": {"docker": docker_settings}}
pipeline_args["config_path"] = os.path.join("configs", "training.yaml")
fe_t_configured = training.with_options(**pipeline_args)

In [11]:
# Lets run the pipeline
fe_t_configured()

Initiating a new run for the pipeline: training.
Reloading configuration file /home/apenner/PycharmProjects/zenml-projects/stack-showcase/.zen/config.yaml
Reusing registered version: (version: 17).
New model version 12 was created.
Building Docker image(s) for pipeline training.
Building Docker image 715803424590.dkr.ecr.eu-central-1.amazonaws.com/zenml:training-model_trainer-sagemaker_step_operator.
- Including user-defined requirements: pyarrow
- Including integration requirements: aws-profile-manager, boto3<=1.26.76, kubernetes, mlflow>=2.1.1,<=2.6.0, mlserver-mlflow>=1.3.3, mlserver>=1.3.3, s3fs>2022.3.0,<=2023.4.0, sagemaker==2.117.0, scikit-learn<1.3
Could not import Azure service connector: No module named 'azure.identity'.
Step 1/11 : FROM zenmldocker/zenml:0.50.0-py3.9
Step 2/11 : WORKDIR /app
Step 3/11 : COPY .zenml_user_requirements .
Step 4/11 : RUN pip install --default-timeout=60 --no-cache-dir  -r .zenml_user_requirements
Step 5/11 : COPY .zenml_integration_requirements 

# Switch to full Sagemaker Stack

Just one command will allow you to switch the full code execution over to sagemaker. No Sagemaker domain knowledge necessary. No setup of VMs or Kubernetes clusters necessary. No maintenance of any infrastructure either.

![Sagemaker local stack](_assets/sagemaker_stack.png)


In [ ]:
# Finally, this is all that needs to be done to fully switch the code to be run fully on sagemaker

!zenml stack set sagemaker-stack

In [ ]:
fe_t_configured()